In [12]:
import os
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as pl

from read_gsheet import get_specific_range


In [ ]:
get_specific_range('unpreparing_stady_data')

In [134]:
df=pd.read_csv('unpreparing_stady_data.csv',header=1, date_format=True, dayfirst=True).drop_duplicates()
df

,Ранг,Спринт,Модули\Спринты,Дата,Утро,Вечер,Доп занятия,Название
0,1,Спринт 1,Спринт 1,26.05.2025,125.0,109.0,NaN,NaN
1,1,Спринт 1,NaN,27.05.2025,124.0,125.0,NaN,NaN
2,1,Спринт 1,NaN,28.05.2025,103.0,NaN,NaN,NaN
3,1,Спринт 1,NaN,29.05.2025,NaN,NaN,NaN,NaN
4,1,Спринт 1,NaN,30.05.2025,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
376,#8,SQL Lab,NaN,NaN,NaN,NaN,NaN,NaN
377,#7,Python Lab,NaN,24.08.2025,NaN,60.0,NaN,NaN
378,#7,Python Lab,NaN,25.08.2025,120.0,11.0,NaN,NaN
379,#6,YandexGPT,NaN,24.08.2025,60.0,60.0,NaN,NaN


In [131]:
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 379 entries, 0 to 380
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ранг            230 non-null    object 
 1   Спринт          230 non-null    object 
 2   Модули\Спринты  28 non-null     object 
 3   Дата            376 non-null    object 
 4   Утро            43 non-null     float64
 5   Вечер           38 non-null     float64
 6   Доп занятия     24 non-null     float64
dtypes: float64(3), object(4)
memory usage: 23.7+ KB


### Приведем все данные по активностям к единому виду

In [136]:
df=df.rename(columns={'Ранг':'rank',
                   'Спринт':'activity_subtype_name',
                    'Дата':'activity_date',
                    'Утро':'morning',
                    'Вечер':'evening',
                    'Доп занятия':'add_study_duration',
                    'Название':'add_study_type'}).drop(columns=['Модули\Спринты','rank'])

In [137]:
df.activity_date=pd.to_datetime(df['activity_date'])

C:\Users\Misterius\AppData\Local\Temp\ipykernel_5004\687405557.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.activity_date=pd.to_datetime(df['activity_date'])


In [169]:
df=df.fillna(0)
df

,activity_subtype_name,activity_date,morning,evening,add_study_duration,add_study_type
0,Спринт 1,2025-05-26 00:00:00,125.0,109.0,0.0,0
1,Спринт 1,2025-05-27 00:00:00,124.0,125.0,0.0,0
2,Спринт 1,2025-05-28 00:00:00,103.0,0.0,0.0,0
3,Спринт 1,2025-05-29 00:00:00,0.0,0.0,0.0,0
4,Спринт 1,2025-05-30 00:00:00,0.0,0.0,0.0,0
...,...,...,...,...,...,...
376,SQL Lab,0,0.0,0.0,0.0,0
377,Python Lab,2025-08-24 00:00:00,0.0,60.0,0.0,0
378,Python Lab,2025-08-25 00:00:00,120.0,11.0,0.0,0
379,YandexGPT,2025-08-24 00:00:00,60.0,60.0,0.0,0


### Перевод данных из полей morning, evening и add_study в отдельные строки. Приведение к формату duration - daypart  

In [248]:
morning_df=df.query('morning != 0 & activity_date !=0')\
    .drop(columns=['add_study_duration','add_study_type','evening'])
morning_df=morning_df.replace('Итоговый проект 1','Проект модуля 1')
morning_df['daypart']='Утро'
morning_df['morning']=morning_df['morning'].astype(int)
morning_df=morning_df.rename(columns={'morning':'duration'})
morning_df.head()

,activity_subtype_name,activity_date,duration,daypart
0,Спринт 1,2025-05-26 00:00:00,125,Утро
1,Спринт 1,2025-05-27 00:00:00,124,Утро
2,Спринт 1,2025-05-28 00:00:00,103,Утро
10,Спринт 1,2025-06-05 00:00:00,77,Утро
13,Спринт 1,2025-06-08 00:00:00,15,Утро


In [246]:
evening_df=df.query('evening != 0 & activity_date !=0')\
    .drop(columns=['add_study_duration','add_study_type','morning'])
evening_df=evening_df.replace('Итоговый проект 1','Проект модуля 1')
evening_df['daypart']='Вечер'
evening_df['evening']=evening_df['evening'].astype(int)
evening_df=evening_df.rename(columns={'evening':'duration'})
evening_df.head()

,activity_subtype_name,activity_date,duration,daypart
0,Спринт 1,2025-05-26 00:00:00,109,Вечер
1,Спринт 1,2025-05-27 00:00:00,125,Вечер
14,Спринт 2,2025-06-09 00:00:00,62,Вечер
16,Спринт 2,2025-06-11 00:00:00,95,Вечер
17,Спринт 2,2025-06-12 00:00:00,123,Вечер


In [184]:
web_df=df.query('add_study_type == "Вебинар"')
web_df=web_df.drop(columns='activity_subtype_name')\
     .rename(columns={'add_study_type':'activity_subtype_name'})
web_df['duration']=web_df['evening']
web_df['daypart']='Вечер'
web_df=web_df.drop(columns=['morning','evening','add_study_duration'])
web_df['duration']=web_df['duration'].astype(int)
web_df

,activity_date,activity_subtype_name,duration,daypart
10,2025-06-05 00:00:00,Вебинар,0,Вечер
36,2025-07-01 00:00:00,Вебинар,75,Вечер
37,2025-07-02 00:00:00,Вебинар,117,Вечер
51,2025-07-16 00:00:00,Вебинар,100,Вечер
65,2025-07-30 00:00:00,Вебинар,110,Вечер
91,2025-08-25 00:00:00,Вебинар,120,Вечер


In [218]:
add_study_df=df.query('add_study_type != 0 & activity_date !=0 & add_study_type != "Вебинар"')
add_study_df=add_study_df.drop(columns='activity_subtype_name')\
    .rename(columns={'add_study_type':'activity_subtype_name','duration':'add_study_duration'})
# add_study_df['duration']=add_study_df['duration'].astype(int)
add_study_df['duration_evening']=add_study_df.add_study_duration
add_study_df['duration_morning']=0
add_study_df.loc[38,'duration_evening']=120
add_study_df.loc[39,'duration_evening']=20
add_study_df.loc[52,'duration_evening']=30
add_study_df.loc[52,'duration_morning']=120
add_study_df.loc[63,'duration_evening']=60
add_study_df.loc[63,'duration_morning']=120
add_study_df.loc[64,'duration_morning']=120
add_study_df.loc[64,'duration_evening']=120
add_study_df.loc[66,'duration_morning']=120
add_study_df.loc[66,'duration_evening']=60
add_study_df.loc[67,'duration_morning']=120
add_study_df.loc[67,'duration_evening']=120
add_study_df.loc[74,'duration_morning']=120
add_study_df.loc[74,'duration_evening']=120
add_study_df.loc[76,'duration_morning']=120
add_study_df.loc[76,'duration_evening']=60
add_study_df.loc[77,'duration_morning']=120
add_study_df.loc[77,'duration_evening']=120
add_study_df.loc[78,'duration_morning']=120
add_study_df.loc[78,'duration_evening']=240
add_study_df.loc[79,'duration_morning']=120
add_study_df.loc[79,'duration_evening']=120
add_study_df.loc[86,'duration_morning']=60
add_study_df.loc[86,'duration_evening']=60

add_study_df.head()

,activity_date,morning,evening,add_study_duration,activity_subtype_name,duration_evening,duration_morning
38,2025-07-03 00:00:00,82.0,0.0,120.0,sql-ex.ru (задачи),120.0,0
39,2025-07-04 00:00:00,0.0,0.0,20.0,sql-ex.ru (задачи),20.0,0
41,2025-07-06 00:00:00,0.0,0.0,46.0,sql-ex.ru (задачи),46.0,0
50,2025-07-15 00:00:00,0.0,0.0,55.0,sql-ex.ru (задачи),55.0,0
52,2025-07-17 00:00:00,0.0,0.0,150.0,lab.karpov.courses,30.0,120


In [244]:
add_study_morning=add_study_df.query('duration_morning != 0')\
    .drop(columns=['morning','evening','add_study_duration','duration_evening'])\
    .rename(columns={'duration_morning':'duration'})
add_study_morning.duration=pd.to_numeric(add_study_morning.duration)
add_study_morning['daypart']='Утро'
add_study_morning

,activity_date,activity_subtype_name,duration,daypart
52,2025-07-17 00:00:00,lab.karpov.courses,120,Утро
63,2025-07-28 00:00:00,lab.karpov.courses,120,Утро
64,2025-07-29 00:00:00,lab.karpov.courses,120,Утро
66,2025-07-31 00:00:00,lab.karpov.courses,120,Утро
67,2025-08-01 00:00:00,lab.karpov.courses,120,Утро
74,2025-08-08 00:00:00,lab.karpov.courses,120,Утро
76,2025-08-10 00:00:00,lab.karpov.courses,120,Утро
77,2025-08-11 00:00:00,lab.karpov.courses,120,Утро
78,2025-08-12 00:00:00,lab.karpov.courses,120,Утро
79,2025-08-13 00:00:00,lab.karpov.courses,120,Утро


In [241]:
add_study_evening=add_study_df.query('duration_evening != 0')\
    .drop(columns=['morning','evening','add_study_duration','duration_morning'])\
    .rename(columns={'duration_evening':'duration'})
add_study_evening['duration']=add_study_evening['duration'].astype(int)
add_study_evening['daypart']='Вечер'
add_study_evening

,activity_date,activity_subtype_name,duration,daypart
38,2025-07-03 00:00:00,sql-ex.ru (задачи),120,Вечер
39,2025-07-04 00:00:00,sql-ex.ru (задачи),20,Вечер
41,2025-07-06 00:00:00,sql-ex.ru (задачи),46,Вечер
50,2025-07-15 00:00:00,sql-ex.ru (задачи),55,Вечер
52,2025-07-17 00:00:00,lab.karpov.courses,30,Вечер
53,2025-07-18 00:00:00,lab.karpov.courses,60,Вечер
60,2025-07-25 00:00:00,lab.karpov.courses,56,Вечер
62,2025-07-27 00:00:00,lab.karpov.courses,60,Вечер
63,2025-07-28 00:00:00,lab.karpov.courses,60,Вечер
64,2025-07-29 00:00:00,lab.karpov.courses,120,Вечер


### Объединяем все очищенные таблицы в одну и сохраняем в .csv

In [250]:
clear_df=pd.concat([morning_df,evening_df,add_study_evening,add_study_morning,web_df])
clear_df
clear_df.to_csv('prep_activities.csv')